# Assistant-02<br/>Assistant Framework

### Get an OpenAI client

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import common
import time
import shelve

## Get an OpenAI client
client = common.get_openai_client()

### Create an assistant

In [10]:
def create_assistant(name,instructions, tools, model):
    assistant = client.beta.assistants.create(
        name=name,
        instructions=instructions,
        tools=tools,
        model=model,
    )
    return assistant

assistant = create_assistant("Math Tutor","You are a personal math tutor. Write and run code to answer math questions.", [{"type": "code_interpreter"}], common.gpt_deployment_name)

In [12]:
def check_if_assistant_exists(assistant_id):
    with shelve.open("assistant_db") as assistant_shelf:
        return assistant_shelf.get(assistant_id, None)
    
def store_assistant(assistant_id):
    with shelve.open("assistant_db", writeback=True) as assistant_shelf:
        assistant_shelf[assistant_id] = assistant_id
    
def check_if_thread_exists(wa_id):
    with shelve.open("threads_db") as threads_shelf:
        return threads_shelf.get(wa_id, None)

def store_thread(wa_id, thread_id):
    with shelve.open("threads_db", writeback=True) as threads_shelf:
        threads_shelf[wa_id] = thread_id

In [13]:
def run_assistant(thread):
    # Run the assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

    # Wait for completion
    while run.status != "completed":
        # Be nice to the API
        time.sleep(0.5)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Retrieve the Messages
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    new_message = messages.data[0].content[0].text.value
    print(f"Generated message: {new_message}")
    return new_message

In [14]:
def generate_response(message_body, wa_id, name):
    
    # Check if there is already a thread_id for the wa_id
    thread_id = check_if_thread_exists(wa_id)

    # If a thread doesn't exist, create one and store it
    if thread_id is None:
        print(f"Creating new thread for {name} with wa_id {wa_id}")
        thread = client.beta.threads.create()
        store_thread(wa_id, thread.id)
        thread_id = thread.id

    # Otherwise, retrieve the existing thread
    else:
        print(f"Retrieving existing thread for {name} with wa_id {wa_id}")
        thread = client.beta.threads.retrieve(thread_id)

    # Add message to thread
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message_body,
    )

    # Run the assistant and get the new message
    new_message = run_assistant(thread)
    print(f"To {name}:", new_message)
    return new_message

In [17]:
# --------------------------------------------------------------
# Test assistant
# --------------------------------------------------------------
new_message = generate_response("What's 2+2", "123", "John")

new_message = generate_response("What's x*x?", "456", "Sarah")

new_message = generate_response("What the value of x, for x=m*y+5, if y=2 and m=-10?", "123", "John")

new_message = generate_response("What is the value of y, for y=x*x?, if x=2?", "456", "Sarah")

Retrieving existing thread for John with wa_id 123
Generated message: 2 + 2 equals 4.
To John: 2 + 2 equals 4.
Retrieving existing thread for Sarah with wa_id 456
Generated message: The expression \( x \times x \) is mathematically represented as \( x^2 \), which is the square of \( x \). If \( x \) is a variable, \( x^2 \) is simply a way to express that variable squared without yet assigning a specific numerical value to it. If you have a particular value of \( x \) in mind, I can calculate that for you. For example, if \( x = 2 \), then \( x \times x = 2 \times 2 = 4 \). Otherwise, \( x^2 \) stands as the algebraic representation of the square of \( x \).
To Sarah: The expression \( x \times x \) is mathematically represented as \( x^2 \), which is the square of \( x \). If \( x \) is a variable, \( x^2 \) is simply a way to express that variable squared without yet assigning a specific numerical value to it. If you have a particular value of \( x \) in mind, I can calculate that fo